In [ ]:
import os
import numpy as np
import pandas as pd
import datetime
import pandas as pd
import matplotlib.pyplot as plt
filelist = os.listdir('measurement')
filelist.sort()
startpoint = np.where(np.array(filelist) == '16157')[0][0]
filelist = filelist[startpoint:]

data = pd.DataFrame()
for fileid in filelist[1:10]:
    df = pd.read_csv('measurement/'+fileid+'/tab_fcd.csv')
    data = pd.concat([data,df])

In [ ]:
df = data[data['ftnode'] =='4697_4694']
df = data.copy()
df2 = df.groupby(['sj']).count()
plt.plot(list(df2.TRAVELID))

In [ ]:
import plotly.express as px
px.set_mapbox_access_token('pk.eyJ1IjoidG9uZ3hpbnJlbiIsImEiOiJjazZnM2phcXEwdTJ5M2pxcHQ3MDRteHNlIn0.ci2XKyZQRC_tAEcvxVIeAQ')

In [ ]:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 15})
df = data.tail(120000)
for ID in df['TRAVELID'].unique()[0:10]:
    df3 = df[df['TRAVELID'] == ID]
    df3 = df3.sort_values('sj')
    df3.index = pd.to_datetime(df3.sj)
    plt.figure(figsize = (15,15))
    plt.subplot(2,1,1)
    plt.plot(df3.longitude,df3.latitude)
    plt.xlabel('longitude')
    plt.ylabel('latitude')
    plt.title('(a)')
    plt.subplot(2,1,2)
    plt.plot(df3.speed)
    plt.xlabel('Time')
    plt.ylabel('Speed')
    plt.title('(b)')
    fig = px.scatter_mapbox(df3, lat="latitude", lon="longitude",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10)
    fig.show()

In [ ]:
plt.hist(data['speed'])

In [ ]:
plt.hist(data[data['speed']>0].speed)

In [ ]:
import os
import numpy as np
import pandas as pd
import datetime
filelist = os.listdir('measurement')
filelist.sort()
startpoint = np.where(np.array(filelist) == '16157')[0][0]
filelist = filelist[startpoint:]
filelist

df = pd.DataFrame()
for fileid in filelist:
    data = pd.read_csv('measurement/'+fileid+'/tab_ldt.csv')
    df = pd.concat([df,data])

In [ ]:
df['Weighted_Speed'] = df['COUNT'] * df['ARITHMETIC_AVERAGE_SPEED']  

# Midnight time is generally measured when there are vehicles in the past, the number of vehicles is small, and the speed is unstable, so only the daytime data is selected, for which the time stamp needs to be modified
def findTime(x):
    return x.time()
def findDate(x):
    return x.date()
df['FROMTIME'] = pd.to_datetime(df['FROMTIME'])
df.index = df['FROMTIME']
df['FROMTIME-Time'] = df['FROMTIME'].apply(findTime) 
df['FROMTIME-Date'] = df['FROMTIME'].apply(findDate) 


# Midnight time is generally measured when there are vehicles in the past, the number of vehicles is small, and the speed is unstable, so only the daytime data is selected, for which the time stamp needs to be modified
def findTime(x):
    return x.time()
def findDate(x):
    return x.date()
df['FROMTIME'] = pd.to_datetime(df['FROMTIME'])
df.index = df['FROMTIME']
df['FROMTIME-Time'] = df['FROMTIME'].apply(findTime)  
df['FROMTIME-Date'] = df['FROMTIME'].apply(findDate)  

In [ ]:
for ID in df["DEVICEID"].unique():
    df1=df[df["DEVICEID"]==ID]
    xianquan = df1.copy() 
    
    xianquan=xianquan[(pd.to_datetime(xianquan["FROMTIME-Time"],
                                      format = '%H:%M:%S')>= pd.to_datetime('07:00:00',format = '%H:%M:%S'))&
                      (pd.to_datetime(xianquan["FROMTIME-Time"],format = '%H:%M:%S')<= pd.to_datetime('12:00:00',format = '%H:%M:%S'))]
    xianquan['FROMTIME'] = pd.to_datetime(xianquan['FROMTIME'])
    xianquan.index = xianquan['FROMTIME']

    interval = 5 
    data_sum= xianquan.resample(datetime.timedelta(seconds=interval *60)).sum()
    data_sum['Average_Speed'] = data_sum['Weighted_Speed'] / data_sum['COUNT']
    data_sum['rho'] = data_sum['FLOW'] / data_sum['Average_Speed']/(interval/60)
    
    plt.figure()
    plt.scatter(data_sum['rho'],data_sum['Average_Speed'])
    plt.xlabel('rho')
    plt.ylabel('Average_Speed')
    plt.title(str(ID))

In [ ]:
import plotly.express as px
px.set_mapbox_access_token('pk.eyJ1IjoidG9uZ3hpbnJlbiIsImEiOiJjazZnM2phcXEwdTJ5M2pxcHQ3MDRteHNlIn0.ci2XKyZQRC_tAEcvxVIeAQ')

In [ ]:
import os
import numpy as np
import pandas as pd
import datetime
import pandas as pd
import matplotlib.pyplot as plt
filelist = os.listdir('measurement')
filelist.sort()
startpoint = np.where(np.array(filelist) == '16157')[0][0]
filelist = filelist[startpoint:]

data = pd.DataFrame()
for fileid in filelist[1:3]:
    df = pd.read_csv('measurement/'+fileid+'/tab_fcd.csv')
    data = pd.concat([data,df])

### Floating car map match

In [ ]:
data['sj'] = pd.to_datetime(data['sj'])
for i in data.TRAVELID.unique()[0:10]:
    df = data[data['TRAVELID'] == i]
    df = df.sort_values('sj')
    df['sj'] = df['sj'].astype(str)
    fig = px.line_mapbox(df, lat="latitude", lon="longitude",zoom=10,hover_data=['sj','speed',])
    fig.show()

    fig = px.line(df,y = 'speed')
    fig.show()

In [ ]:
df = data[data.sj.isin(data.sj.unique()[0:3])]
fig = px.scatter_mapbox(df, lat="latitude", lon="longitude",color = 'sj',zoom=10,hover_data=['TRAVELID','speed','ftnode'])
fig.show()

### number of vehicles on each road section

In [ ]:
df = data[data.sj.isin(data.sj.unique()[0:1])]
fig = px.scatter_mapbox(df, lat="latitude", lon="longitude",color = 'ftnode',zoom=10,hover_data=['TRAVELID','speed','ftnode'])
fig.show()